In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from glob import glob
import pandas as pd
import pickle
from torch.utils.data import RandomSampler
import random
import scipy
import torch.nn.functional as F
from PIL import Image
from glob import glob
import wandb
import re
from adjustText import adjust_text
import seaborn as sns
import nibabel as nib
import scipy
import statannot
import argparse
import sys
#print(glob('../../*'))
sys.path.append('../../modified_medsam_repo')
from MedSAM_HCP.utils_hcp import *
from MedSAM_HCP.dataset import MRIDataset, MRIDatasetForPooled, MRIDataset_Imgs, load_datasets, LabelConverter
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn


/gpfs/home/kn2347/.conda/envs/medsam/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_model(model_type, model_path, num_classes):
    result = torch.load(model_path)
    try:
        if 'model' in result.keys():
            splits = model_path.split('/')
            new_path = os.path.join('/'.join(splits[:-1]), f'{splits[-1].split(".pth")[0]}_sam_readable.pth')
            print(f'model path converted to sam readable format and saved to {new_path}')

            result = result['model']

            # now remove the "module." prefix
            result_dict = {}
            for k,v in result.items():
                key_splits = k.split('.')
                assert key_splits[0] == 'module'
                new_k = '.'.join(key_splits[1:])
                result_dict[new_k] = v

            torch.save(result_dict, new_path)
            model_path = new_path

    except (AttributeError):
        # already in the correct format
        print('model path in readable format already')

    if model_type == 'multitask_unprompted':
        model = build_sam_vit_b_multiclass(num_classes, checkpoint=model_path).to('cuda')
    elif model_type == 'pooltask_yolov7_prompted':
        model = build_sam_vit_b_multiclass(num_classes, checkpoint=model_path).to('cuda')
    elif model_type == 'singletask_unet':
        model = torch.load(model_path)
    else:
        # singletask model
        model = build_sam_vit_b_multiclass(3, checkpoint=model_path).to('cuda')

    model.eval()
    return model


In [3]:
df_hcp = pd.read_csv('/gpfs/home/kn2347/HCP_MedSAM_project/modified_medsam_repo/hcp_mapping_processed.csv')
df_desired = pd.read_csv('/gpfs/home/kn2347/HCP_MedSAM_project/modified_medsam_repo/darts_name_class_mapping_processed.csv')
label_converter = LabelConverter(df_hcp, df_desired)

In [5]:
df_hcp = pd.read_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/path_df_unet.csv', dtype={'id': str})
df_brats = pd.read_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/other_path_dfs/path_df_brats_unet.csv')

In [51]:
preprocess_input


functools.partial(<function preprocess_input at 0x15545ab54c10>, input_space='RGB', input_range=[0, 1], mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [8]:
label = 1
preprocess_input = get_preprocessing_fn('resnet18', pretrained='imagenet')
model_path = glob(f'/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/unet_retrain_all_labels_9-9-24/training/{label}/*-best_model.pth')[0]
model = load_model('singletask_unet', model_path, num_classes=1)
dataset_hcp = MRIDataset_Imgs(df_hcp, label_id = label, bbox_shift=0, label_converter = label_converter, NUM_CLASSES=2, as_one_hot=True, pool_labels=False, preprocess_fn=preprocess_input)
dataset_brats = MRIDataset_Imgs(df_brats, label_id = label, bbox_shift=0, label_converter = label_converter, NUM_CLASSES=2, as_one_hot=True, pool_labels=False, preprocess_fn=preprocess_input)

model path in readable format already


In [63]:
dataset_hcp[200][0].numpy().max()

1.2108061

In [64]:
dataset_brats[128][0].numpy().max()

2.64

In [83]:
df_hcp.id.unique()

array(['100206', '100307', '100408', ..., '994273', '995174', '996782'],
      dtype=object)

In [79]:
#df_brats.iloc[128,2]
df_hcp.iloc[128,2]

for id in df_hcp.id.unique():
    listo = []
    for i in range(256):
        npo = np.array(Image.open(f'/gpfs/data/cbi/hcp/hcp_ya/hcp_ya_slices_npy/dir_structure_for_yolov7/train/images/{id}_slice{i}.png'))
        listo.append(npo.max())
    print(f'{id}: {np.max(listo)}')



In [73]:
npo = np.array(Image.open('/gpfs/data/luilab/karthik/MICCAI_BraTS2020_TrainingData/images/images001_slice128.png'))
print(preprocess_input(npo).max())


2.6399999999999997
